#### Import Dependencies

In [28]:
import os
import time
import json
from datetime import datetime
import requests
from dotenv import load_dotenv
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, from_json, udf, size, array_sort, split, when, lit, 
    regexp_replace, expr, concat_ws, to_date, monotonically_increasing_id
)
from pyspark.sql.types import *

#### Fetch Movies Data 

In [ ]:
def fetch_movie_data_spark(movie_ids, save_path="data/raw"):
    # Load env variables
    load_dotenv()
    api_key = os.getenv('API_KEY')
    base_url = os.getenv('BASE_url') 

    # Initialize Spark
    spark = SparkSession.builder.appName("FetchTMDBMovies").getOrCreate()

    movies_data = []
    for movie_id in movie_ids:
        for attempt in range(3):
            try:
                url = f'{base_url}{movie_id}?api_key={api_key}&append_to_response=credits'
                r = requests.get(url)
                if r.status_code == 200:
                    movies_data.append(r.json())
                    break
                elif r.status_code == 429:
                    print("Rate limit hit. Waiting 3 seconds...")
                    time.sleep(3)
                else:
                    print(f"Failed: {movie_id} (Status {r.status_code})")
                    time.sleep(1)
            except Exception as e:
                print(f"Error: {e}")
                time.sleep(1)

        time.sleep(0.3)  # ~3 req/sec max

    # Convert to RDD and DataFrame
    rdd = spark.sparkContext.parallelize([json.dumps(movie) for movie in movies_data])
    df = spark.read.json(rdd)


    # Save using timestamp
    timestamp = datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
    full_path = f"{save_path}/moviesData_{timestamp}.parquet"
    df.coalesce(1).write.mode("overwrite").parquet(full_path)

    print(f"Data saved to {full_path}")

    return df


In [ ]:
#calling the fetch_movie_data_spark to do data extraxtions
movie_ids = [
    0, 299534, 19995, 140607, 299536, 597, 135397, 420818, 24428,
    168259, 99861, 284054, 12445, 181808, 330457, 351286, 109445,
    321612, 260513
]

df = fetch_movie_data_spark(movie_ids)

#df.show(3, truncate=False)

#### Data Cleaning

In [38]:
# Spark session
spark = SparkSession.builder.appName("TMDBCleaning").getOrCreate()

def dataCleaning(df):
    # Drop unnecessary columns
    drop_cols = ["adult", "imdb_id", "original_title", "video", "homepage"]
    df = df.drop(*drop_cols)

    # Parse nested JSON fields
    #genre
    df = df.withColumn("genres", concat_ws("|", expr("transform(genres, x -> x.name)")))

    #belongs_to_collection
    df = df.withColumn("belongs_to_collection", col("belongs_to_collection.name"))

    #production_countries
    df = df.withColumn("production_countries", expr("array_join(transform(production_countries, x -> x.name), '|')"))

    #production_companies
    df = df.withColumn("production_companies", expr("array_join(transform(production_companies, x -> x.name), '|')"))

    #spoken_languages
    df = df.withColumn("spoken_languages", expr("array_join(transform(spoken_languages, x -> x.english_name), '|')"))

    # Extract cast, director, cast size, and crew size from credits
    # Extract relevant info
    df = df.withColumn("cast", expr("array_join(transform(credits.cast, x -> x.name), '|')"))
    df = df.withColumn("cast_size", size("credits.cast"))
    df = df.withColumn("crew_size", size("credits.crew"))

    # Safely extract director — check for empty array
    df = df.withColumn("director", expr("""
        CASE 
            WHEN size(filter(credits.crew, x -> x.job = 'Director')) > 0 
            THEN filter(credits.crew, x -> x.job = 'Director')[0].name 
            ELSE NULL 
        END
    """))

    # Drop the original credits field
    df = df.drop("credits")

    # Clean up multiple-value columns (sorted order)
    for col_name in ["genres", "production_countries", "spoken_languages"]:
        df = df.withColumn(
            col_name,
            expr(f"array_join(array_sort(transform(split({col_name}, '\\\\|'), x -> trim(x))), '|')")
        )

    # Convert numeric fields
    num_cols = ["budget", "popularity", "id", "revenue", "runtime"]
    for field in num_cols:
        df = df.withColumn(field, col(field).cast("double"))

    df = df.withColumn("release_date", to_date("release_date", "yyyy-MM-dd"))

    # Replace zero with null
    for col_name in ["budget", "revenue", "runtime"]:
        df = df.withColumn(col_name, when(col(col_name) == 0, None).otherwise(col(col_name)))

    # Add new columns
    df = df.withColumn("budget_musd", col("budget") / 1e6)
    df = df.withColumn("revenue_musd", col("revenue") / 1e6)

    # Vote count zero cleanup
    df = df.withColumn("vote_average", when(col("vote_count") == 0, None).otherwise(col("vote_average")))

    # Missing data handling
    df = df.withColumn("overview", when(col("overview").isin("No Data", "", "nan"), None).otherwise(col("overview")))
    df = df.withColumn("tagline", when(col("tagline").isin("No Data", "", "nan"), None).otherwise(col("tagline")))

    # Drop duplicates and filter final rows
    df = df.dropDuplicates()
    df = df.dropna(subset=["id", "title"])
    important_cols = [
        "id", "title", "tagline", "release_date", "genres", "belongs_to_collection",
        "original_language", "budget_musd", "revenue_musd", "production_companies",
        "production_countries", "vote_count", "vote_average", "popularity", "runtime",
        "overview", "spoken_languages", "cast", "cast_size", "director", "crew_size"
    ]
    df = df.withColumn(
        "non_nulls",
        expr(f"size(filter(array({','.join(important_cols)}), x -> x is not null))")
    )
    df = df.filter(col("non_nulls") >= 10).drop("non_nulls")
    df = df.filter(col("status") == "Released").drop("status")

    # Final columns
    df = df.select([
        "id", "title", "tagline", "release_date", "genres", "belongs_to_collection",
        "original_language", "budget_musd", "revenue_musd", "production_companies",
        "production_countries", "vote_count", "vote_average", "popularity", "runtime",
        "overview", "spoken_languages", "poster_path", "cast", "cast_size", "director", "crew_size"
    ])

    return df.show(5, truncate=False)

In [ ]:
#calling the dataCleaning to do data cleaning
# Load extracted data
df = spark.read.parquet("data/raw/moviesData_*.parquet")
dataCleaning(df)


+--------+-----------------------+-----------------------------+------------+-----------------------------------------+-----------------------------------+-----------------+-----------+------------+-----------------------------------------+------------------------+----------+------------+----------+-------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------+--------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------